# Ensemble of the results

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
model = '{image_model}'
tabular_model = '{tabular_model}'

In [ ]:
images_path = os.path.join('..', '..', 'images')

In [ ]:
all_results = pd.DataFrame()

In [ ]:
def get_metrics(y_test, y_pred):
    print('MCC', metrics.matthews_corrcoef(y_test, y_pred))
    print('F1-score', metrics.f1_score(y_test, y_pred))
    print('ROC-AUC', metrics.roc_auc_score(y_test, y_pred))
    cm = metrics.confusion_matrix(y_test, y_pred)
    metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
def get_cv_metrics(y_test, merge_cols, merge_func, roc_values):
    results = pd.DataFrame()
    cms = []
    tpr_rates = []
    mean_fpr = np.linspace(0, 1, 100)
    for fold in range(1, 3+1):
        df = pd.read_csv(f'predictions_{tabular_model}_{model}_fold_{fold}.csv')
        df['Tabular_y_proba'] = df.Tabular_y_pred_proba
        y_pred, y_proba = merge_func(df, merge_cols)
        y_test = df.Tabular_y_test
        tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
        cm = metrics.confusion_matrix(y_test, y_pred)
        cms.append(cm)
        fpr_proba, tpr_proba, threshold_proba = metrics.roc_curve(y_test, y_proba)
        # plt.plot(fpr_proba, tpr_proba, label=f'Fold {fold} - ROC AUC = {roc_score}', alpha=0.3,)
        interp_tpr = np.interp(mean_fpr, fpr_proba, tpr_proba)
        interp_tpr[0] = 0.0
        tpr_rates.append(interp_tpr)
        res =  {
          'models' : [merge_cols],
          'ensemble_func' : merge_func.__name__.split('_')[1],
          'fold' : fold,
          'balanced_accuracy' : metrics.balanced_accuracy_score(y_test, y_pred),
          'roc_auc_score' : [metrics.roc_auc_score(y_test, y_proba)],
          'recall_weighted' : [metrics.recall_score(y_test, y_pred, average='weighted')], 
          'f1_weighted' : [metrics.f1_score(y_test, y_pred, average='weighted')], 
          'PPV_precision_weighted' : [metrics.precision_score(y_test, y_pred, average='weighted')], 
          'matthews_corrcoef' : [metrics.matthews_corrcoef(y_test, y_pred)],
          'specificity' : [tn / (tn+fp)],
          'NPV' : [tn / (tn + fn)],
          'tp' : [tp],
          'fp' : [fp],
          'fn' : [fn],
          'tn' : [tn]
        }
        results = pd.concat([results, pd.DataFrame(res)], ignore_index=True)
    mean_tpr = np.mean(tpr_rates, axis=0)
    roc_values[f'{merge_cols} - {merge_func.__name__.split("_")[1].upper()}'] = {
        'mean_tpr' : mean_tpr,
        'mean_fpr' : mean_fpr,
        'std_roc' : results.roc_auc_score.std().round(3),
        'mean_roc' : results.roc_auc_score.mean().round(3),
    }

    # plot roc_curve
    sns.set(font_scale=1.5)
    plt.figure(figsize=(15, 15))
    sns.set_style("whitegrid")
    plt.plot(
        mean_fpr, 
        mean_tpr, 
        label=f'Mean ROC (AUC = {results.roc_auc_score.mean().round(3)} $\pm$ {results.roc_auc_score.std().round(3)})',
        lw=5
    )
    plt.legend(fontsize='14')
    plt.plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color = 'g', label='Random Classifier')
    
    plt.xlabel('False Positive Rate', fontdict=dict(size=20))
    plt.ylabel('True Positive Rate', fontdict=dict(size=20))
    plt.title(f'ROC Curves', fontdict=dict(size=25))
    plt.savefig(os.path.join(images_path, f'roc_curve_ensemble_{merge_cols}_{merge_func.__name__}.png'))
    
    # plot CM Matrix
    mean_cm = np.array(cms).mean(axis=0)
    group_counts = ['{0:0.0f}'.format(value) for value in
                mean_cm.flatten()]
    percentages_cm = (mean_cm.T / mean_cm.sum(axis=1)).T
    group_percentages = ['{0:.2%}'.format(value) for value in
                     percentages_cm.flatten()]
    labels = [f'{v1}\n({v2})' for v1, v2 in
          zip(group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    
    plt.figure(figsize=(15, 10))
    sns.set(font_scale=2.5)
    sns.heatmap(
        mean_cm, 
        annot=labels, 
        fmt='', 
        cmap='Blues', 
        xticklabels=['Vaginal Delivery', 'Cesarean Delivery'], 
        yticklabels=['Vaginal Delivery', 'Cesarean Delivery'])
    
    plt.xlabel('Predicted Label', fontdict=dict(size=25))
    plt.ylabel('True Label', fontdict=dict(size=25))
    plt.savefig(os.path.join(images_path, f'average_cm_{merge_cols}_{merge_func.__name__.split("_")[1]}.png'), transparent=True)
    return results, roc_values


In [ ]:
roc_values = {}

In [ ]:
def select_max(df, merge_cols):
    y_pred_cols = [f'{col}_y_pred' for col in merge_cols]
    y_pred_proba_cols = [f'{col}_y_proba' for col in merge_cols]
    return df[y_pred_cols].max(axis=1), df[y_pred_proba_cols].max(axis=1)

def select_average(df, merge_cols):
    cols = [f'{col}_y_proba' for col in merge_cols]
    mean = df[cols].mean(axis=1)
    preds = (mean > 0.5).astype(int)
    return preds, mean

In [ ]:
merge_cols = ['Femur']
df, roc_values = get_cv_metrics('Tabular_y_test', merge_cols, select_max, roc_values)
all_results = pd.concat([all_results, df], ignore_index=True)

In [ ]:
merge_cols = ['Femur']
df, roc_values = get_cv_metrics('Tabular_y_test', merge_cols, select_average, roc_values)
all_results = pd.concat([all_results, df], ignore_index=True)

In [ ]:
merge_cols = ['Tabular', 'Head', 'Femur', 'Abdomen']
df, roc_values = get_cv_metrics('Tabular_y_test', merge_cols, select_average, roc_values)
all_results = pd.concat([all_results, df], ignore_index=True)

In [ ]:
merge_cols = ['Tabular', 'Head', 'Femur', 'Abdomen']
df, roc_values = get_cv_metrics('Tabular_y_test', merge_cols, select_max, roc_values)
all_results = pd.concat([all_results, df], ignore_index=True)

In [ ]:
merge_cols = ['Femur', 'Abdomen', 'Head']
df, roc_values = get_cv_metrics('Tabular_y_test', merge_cols, select_average, roc_values)
all_results = pd.concat([all_results, df], ignore_index=True)

In [ ]:
merge_cols = ['Femur', 'Abdomen', 'Head']
df, roc_values = get_cv_metrics('Tabular_y_test', merge_cols, select_max, roc_values)
all_results = pd.concat([all_results, df], ignore_index=True)

In [ ]:
merge_cols = ['Tabular', 'Femur']
df, roc_values = get_cv_metrics('Tabular_y_test', merge_cols, select_max, roc_values)
all_results = pd.concat([all_results, df], ignore_index=True)

In [ ]:
merge_cols = ['Tabular', 'Femur']
df, roc_values = get_cv_metrics('Tabular_y_test', merge_cols, select_average, roc_values)
all_results = pd.concat([all_results, df], ignore_index=True)

In [ ]:
all_results.models = all_results.models.astype(str)

In [ ]:
fold = all_results.pop('fold')
mean = all_results.groupby(['models', 'ensemble_func']).mean()
std = all_results.groupby(['models', 'ensemble_func']).std()
mean = mean.add_prefix('mean_')
std = std.add_prefix('std_')
final_results = pd.concat([mean, std], axis=1)
final_results = final_results.reset_index()

In [ ]:
final_results.to_csv(f'results_ensemble_{model}_{tabular_model}.csv', index=False, sep=';')

In [ ]:
roc_values = dict(sorted(roc_values.items(), key=lambda x: x[1]['mean_roc'], reverse=True))

In [ ]:
sns.set(font_scale=1.5)
plt.figure(figsize=(15, 15))
sns.set_style('whitegrid')
for ensemble_model, values in roc_values.items():
    if ensemble_model == "['Tabular'] - MAX":
        ensemble_model = 'AdaBoostClassifier'
    if ensemble_model == "['Femur'] - MAX":
        ensemble_model = 'Inception Femur'
    plt.plot(
        values['mean_fpr'], 
        values['mean_tpr'], 
        label=f"Mean ROC (AUC = {values['mean_roc']} $\pm$ {values['std_roc']}) - {ensemble_model}",
        lw=3
    )
plt.plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color = 'g', label='Random Classifier')
plt.legend(fontsize='14')

plt.xlabel('False Positive Rate', fontdict=dict(size=20))
plt.ylabel('True Positive Rate', fontdict=dict(size=20))
plt.title(f'ROC Curves', fontdict=dict(size=25))
plt.savefig(os.path.join(images_path, f'roc_curve_ensemble_all.png'))